In [6]:
import numpy as np

In [7]:
# 4 vals per year
# - missing 1: fill with median of that county at that year
# - missing 2: fill with median of that county at that year
# - missing 3: fill with median of that county at that year
# median over mean:
# - the median is less affected by outliers and skewed data than the mean

In [8]:
data = pd.read_csv("../../data/raw/environment data - with counties.csv")

In [9]:
relevant_cols = ["NAME", "LATITUDE", "LONGITUDE", "COUNTY", "ELEVATION", "DATE", "CDSD", "EMXP", "PRCP", "CLDD", "DT00", "DT32", "DX32", "DX70", "DX90", "EMNT", "EMXT", "FZF0", "FZF1", "FZF2", "FZF3", "FZF4", "FZF5", "FZF6", "FZF7", "FZF8", "FZF9", "HTDD", "TAVG", "TMAX", "TMIN", "SNOW", "PSUN"]
# consider whether emnt_attribute is needed for the date down the line
target_cols = []

In [10]:
df = data.copy()
relevant = df[relevant_cols].drop_duplicates()
focused = df[target_cols]

# categorize columns
categorical_cols = list(relevant.select_dtypes("object").columns)
#id_cols = [col for col in training.columns if col.endswith("_id")]
id_cols = ["ELEVATION", "DATE"]
target_col = []
exclude_cols = categorical_cols + id_cols + target_col
numerical_cols = [col for col in relevant.columns if col not in exclude_cols]

# categorize columns for future use
categorical_cols = list(relevant.select_dtypes("object").columns)
numerical_cols = [col for col in relevant.columns if col not in categorical_cols]

In [11]:
relevant = relevant.sort_values(by=["COUNTY", "DATE"]).reset_index(drop=True).drop(["NAME", "LATITUDE", "LONGITUDE", "ELEVATION"], axis=1)

In [12]:
relevant_groupby = relevant.groupby(["COUNTY", "DATE"])

for county in relevant["COUNTY"].unique():
    county_data = relevant[relevant["COUNTY"] == county]
    for year in county_data["DATE"].unique():
        # get current slice of data from that year for that county
        current_group = relevant_groupby.get_group((county, year))
        # get the names of the columns that are missing all their data
        missing_cols = current_group.loc[:, current_group.isna().sum() == current_group.shape[0]].columns
        # get the names of the columns that have some missing data but not all
        other_cols = [col for col in current_group.columns if col not in missing_cols]
        # fill in the missing values in the columns in other_cols with their respective medians
        current_group[other_cols] = current_group[other_cols].fillna(current_group[other_cols].median())
        # save values in current group back into original dataframe ('relevant')
        relevant.update(current_group, overwrite=False)

In [31]:
# SPLIT THE DATA INTO TWO - ONE FOR PREDICTIONS, ONE FOR CLUSTERING
clustering = relevant.copy()
predictions = relevant.copy()
for col in predictions.columns:
    missing_percentage = relevant[col].isna().sum() / predictions.shape[0] * 100
    # set a threshhold to drop columns
    if missing_percentage < 50.0:
        predictions[col] = predictions[col].interpolate(method="linear")
    # if it exceeds it, drop the column?
    else:
        predictions = predictions.drop(columns=[col], axis=1)

clustering = clustering.fillna(-9999999) # WIP
predictions = predictions.fillna(predictions.median()).groupby(["COUNTY", "DATE"]).median().reset_index()

In [32]:
predictions.head()

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF5,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW
0,Addison,1990,314.0,2.520,48.940,314.0,2.0,136.0,55.0,107.0,...,24.0,24.0,24.0,17.0,11.0,7198.0,47.00,54.20,38.50,69.40
1,Addison,1991,314.0,1.900,37.150,314.0,30.0,161.0,63.0,109.0,...,30.0,27.0,20.0,20.0,11.0,7941.5,42.90,53.60,32.30,57.70
2,Addison,1992,209.0,1.660,34.510,209.0,28.0,174.0,58.0,105.0,...,31.0,26.0,24.0,15.0,15.0,7909.5,43.90,53.40,31.40,22.50
3,Addison,1993,264.5,1.925,39.240,264.5,42.0,192.0,70.5,105.5,...,30.5,28.0,21.5,19.5,9.5,8435.5,41.25,53.10,29.40,129.95
4,Addison,1994,325.5,1.520,40.745,325.5,42.0,180.5,69.0,102.0,...,30.5,24.5,24.0,18.5,10.0,8163.0,41.70,53.25,30.15,79.45


In [33]:
clustering.to_csv("../../data/cleaned/environment/environment clustering.csv", index=False)
predictions.to_csv("../../data/cleaned/environment/environment predictions.csv", index=False)

In [ ]:
relevant.to_csv("../../data/cleaned/environment/cleaned environment data.csv", index=False)